In [8]:
%reload_ext autoreload
%autoreload 2

# 0.04: Dataloader Speed Test

We test the speed of the dataloader to determine the optimal number of workers.

In [2]:
import torch
import numpy as np
from quantile_diffusion_mia.dataset import QuantileRegressionDataset
from quantile_diffusion_mia.config import DATASET_CONFIG, MODEL_CONFIG
from torch.utils.data import DataLoader

dataset_name = "CIFAR10"
config = DATASET_CONFIG[dataset_name]

# Load the train indices
split_path = "../" + config['quantile_regression_split_path']
output = np.load(split_path)
quantile_train_indices = output['quantile_train_indices']

# Load the dataset
data_path = "../" + config['quantile_regression_data_path']
train_dataset = QuantileRegressionDataset.load(data_path)
train_dataset.set_indices(quantile_train_indices)

In [ ]:
import time

def test_num_workers(dataset, batch_size=64, num_worker_values=[0, 2, 4, 8, 16]):
    timings = {}
    for num_workers in num_worker_values:
        loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
        start_time = time.time()
        for _ in range(5):  # Load a few batches to get an average time
            _ = next(iter(loader))
        timings[num_workers] = time.time() - start_time
        print(f'num_workers={num_workers}, Time: {timings[num_workers]:.4f}s')
    return timings

# Usage: pass your dataset to test_num_workers function
timings = test_num_workers(train_dataset)

num_workers=0, Time: 0.0357s
num_workers=2, Time: 0.6411s
num_workers=4, Time: 0.8507s
num_workers=8, Time: 1.2934s
num_workers=16, Time: 2.6478s
